In [1]:
import simuOpt
simuOpt.setOptions(alleleType='short', quiet=True, numThreads=4)
import simuPOP as sim
import numpy as np
import pandas as pd
import random
import h5py
from saegus import analyze, breed, operators, parameters, parse

In [2]:
np.set_printoptions(precision=3, suppress=True)

In [3]:
example_pop = sim.loadPopulation('example_pop.pop')

In [4]:
example_pop.addInfoFields(['ind_id', 'mother_id', 'father_id', 'g', 'p'])

In [5]:
sim.tagID(example_pop)

In [6]:
tf = parse.TusonFounders()

In [7]:
recom_map = tf.parse_recombination_rates('genetic_map.txt')

In [8]:
founders = [[1, 2], [3, 4], [5, 6], [7, 8]]

In [9]:
offspring_per_pair = 250

In [10]:
magic = breed.MAGIC(example_pop, founders, recom_map)

In [11]:
example_pop.popSize()

105

In [12]:
magic.generate_f_one(founders, offspring_per_pair)

In [13]:
random_cross = breed.RandomCross(example_pop, 4, 250)

In [14]:
mothers, fathers = random_cross.converging_random_cross()

In [15]:
chooser = breed.SecondOrderPairIDChooser(mothers, fathers)

In [16]:
example_pop.evolve(
    matingScheme=sim.HomoMating(
        sim.PyParentsChooser(chooser.snd_ord_id_pairs),
        sim.OffspringGenerator(ops=[
            sim.IdTagger(),
            sim.PedigreeTagger(),
            sim.Recombinator(rates=recom_map)], numOffspring=1),
    subPopSize=1000),
    gen=1,
)

1

In [17]:
final_random_cross = breed.RandomCross(example_pop, 2, 500)
mothers, fathers = final_random_cross.converging_random_cross()

In [18]:
final_chooser = breed.SecondOrderPairIDChooser(mothers, fathers)

In [19]:
example_pop.evolve(
    matingScheme=sim.HomoMating(
        sim.PyParentsChooser(final_chooser.snd_ord_id_pairs),
        sim.OffspringGenerator(ops=[
            sim.IdTagger(),
            sim.PedigreeTagger(),
            sim.Recombinator(rates=recom_map)], numOffspring=1),
    subPopSize=1000),
    gen=1,
)

1

In [20]:
sim.stat(example_pop, numOfSegSites=sim.ALL_AVAIL, vars=['segSites'])

In [21]:
qtl = sorted(random.sample(example_pop.dvars().segSites, 20))

In [22]:
qtl

[3368,
 3465,
 5653,
 6164,
 8651,
 13037,
 13104,
 16368,
 20018,
 22591,
 23921,
 24955,
 25956,
 31657,
 33679,
 33684,
 33709,
 34102,
 36196,
 37192]

In [23]:
sim.stat(example_pop, alleleFreq=sim.ALL_AVAIL)
allele_data = analyze.gather_allele_data(example_pop)

In [24]:
trait = parameters.Trait()

In [25]:
ae_table = trait.construct_allele_effects_table(example_pop, qtl, random.expovariate, 1)

In [ ]:
ae_table

In [26]:
ae_array = trait.construct_ae_array(ae_table, qtl)

In [ ]:
ae_array[qtl]

In [28]:
integrated_example_data = h5py.File("example_integrated_data.hdf5")

In [29]:
allele_group = integrated_example_data.create_group('allele')
genotype_group = integrated_example_data.create_group('genotype')
trait_group = integrated_example_data.create_group('trait')

In [30]:
operators.calculate_g(example_pop, ae_array)

In [ ]:
print(np.array(example_pop.indInfo('g')))

In [31]:
operators.calculate_error_variance(example_pop, 0.7)

In [32]:
example_pop.dvars().epsilon

7.388681037991911

In [ ]:
operators.calculate_p(example_pop)

In [ ]:
np.mean(example_pop.indInfo('g')), np.var(example_pop.indInfo('g'))

In [ ]:
np.mean(example_pop.indInfo('p')), np.var(example_pop.indInfo('p'))

In [ ]:
g = np.array(example_pop.indInfo('g'))

In [33]:
example_pop.dvars().gen = 1

In [ ]:
example_pop.evolve(
    preOps=[
        sim.Stat(alleleFreq=sim.ALL_AVAIL),
        sim.Stat(genoFreq=sim.ALL_AVAIL),
        operators.HDF5AlleleFrequencies(allele_group, allele_data),
        operators.HDF5GenotypeFrequencies(genotype_group),
        operators.GenoAdditiveArray(qtl, ae_array),
        operators.PhenoAdditive(),
        operators.HDF5Trait('g', trait_group),
        operators.HDF5Trait('p', trait_group),
    ],
    matingScheme=sim.RandomMating(
        ops=[
            sim.IdTagger(),
            sim.PedigreeTagger(),
            sim.Recombinator(rates=recom_map)
        ]),
    gen=10,
)

In [ ]:
integrated_example_data.close()

In [ ]:
example_pop.dvars().genoFreq[0]

In [ ]:
reopened_data = h5py.File('example_integrated_data.hdf5')

In [ ]:
list(reopened_data['genotype'])

In [ ]:
reopened_data.close()

In [ ]:
list(integrated_example_data)

In [ ]:
list(integrated_example_data['trait'])

In [ ]:
np.array(integrated_example_data['trait/3/g']), np.array(integrated_example_data['trait/3/p'])

In [ ]:
np.array(integrated_example_data['allele/7'])

In [ ]:
genofrqs2 = np.array(integrated_example_data['genotype/4'])

In [ ]:
np.sum(genofrqs[])

In [ ]:
genofrqs2

In [ ]:
genokeyz = list(example_pop.dvars().genoFreq[0].keys())

In [ ]:
genofreqs = np.array(genotype_group['7'])

In [ ]:
for gk in genokeyz:
    print(genofreqs[0][gk])

In [ ]:
genofreqs[0]

In [ ]:
genofreqs[0][1, 4]

In [ ]:
np.ndarray((44445, 5, 5))